# Analyze Product Sentiment

In [2]:
!pip install turicreate

     |████████████████████████████████| 91.9MB 67kB/s 
     |████████████████████████████████| 2.1MB 56.7MB/s 
     |████████████████████████████████| 86.3MB 51kB/s 
     |████████████████████████████████| 20.2MB 1.3MB/s 
     |████████████████████████████████| 3.4MB 44.7MB/s 
     |████████████████████████████████| 3.8MB 51.6MB/s 
     |████████████████████████████████| 450kB 51.6MB/s 
ERROR: fastai 0.7.0 has requirement torch<0.4, but you'll have torch 1.4.0 which is incompatible.
ERROR: tensorboard 2.0.2 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
  Found existing installation: Pillow 4.3.0
    Uninstalling Pillow-4.3.0:
      Successfully uninstalled Pillow-4.3.0
  Found existing installation: google-auth-oauthlib 0.4.0
    Uninstalling google-auth-oauthlib-0.4.0:
      Successfully uninstalled google-auth-oauthlib-0.4.0
  Found existing installation: tensorboard 2.1.0
    Uninstalling tensorboard-2.1.0:
      Successfully uninstalled tensor

In [4]:
import turicreate
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Read product review data

In [5]:
products = turicreate.SFrame('gdrive/My Drive/uwml/amazon_baby.sframe')

# Explore data

In [6]:
products

name,review,rating
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0


In [7]:
products.groupby('name',operations={'count':turicreate.aggregate.COUNT()}).sort('count',ascending=False)

name,count
Vulli Sophie the GiraffeTeether ...,785
"Simple Wishes Hands-FreeBreastpump Bra, Pink, ...",562
Infant Optics DXR-5 2.4GHz Digital Video Baby ...,561
Baby Einstein Take AlongTunes ...,547
Cloud b TwilightConstellation Night ...,520
"Fisher-Price BoosterSeat, Blue/Green/Gray ...",489
Fisher-Price RainforestJumperoo ...,450
"Graco Nautilus 3-in-1 CarSeat, Matrix ...",419
Leachco Snoogle TotalBody Pillow ...,388
"Regalo Easy Step WalkThru Gate, White ...",374


# Examine the reivews for the most-reviewed product

In [8]:
giraffe_reviews = products[products['name']=='Vulli Sophie the Giraffe Teether']

In [9]:
giraffe_reviews

name,review,rating
Vulli Sophie the GiraffeTeether ...,He likes chewing on allthe parts especially the ...,5.0
Vulli Sophie the GiraffeTeether ...,My son loves this toy andfits great in the diaper ...,5.0
Vulli Sophie the GiraffeTeether ...,There really should be alarge warning on the ...,1.0
Vulli Sophie the GiraffeTeether ...,All the moms in my moms'group got Sophie for ...,5.0
Vulli Sophie the GiraffeTeether ...,I was a little skepticalon whether Sophie was ...,5.0
Vulli Sophie the GiraffeTeether ...,I have been reading aboutSophie and was going ...,5.0
Vulli Sophie the GiraffeTeether ...,My neice loves her sophieand has spent hours ...,5.0
Vulli Sophie the GiraffeTeether ...,What a friendly face!And those mesmerizing ...,5.0
Vulli Sophie the GiraffeTeether ...,We got this just for myson to chew on instea ...,5.0
Vulli Sophie the GiraffeTeether ...,"My baby seems to likethis toy, but I could ...",3.0


In [10]:
len(giraffe_reviews)

785

In [ ]:
giraffe_reviews['rating'].show()

Materializing SArray...

Done.

Materializing SArray...

Done.

# Building a sentiment classifier

## Build word count vectors

In [11]:
products['word_count'] = turicreate.text_analytics.count_words(products['review'])

In [12]:
products

name,review,rating,word_count
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0,"{'and': 5.0, 'stink':1.0, 'months': 1.0, ..."
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'and': 3.0, 'love': 1.0,'it': 3.0, 'highly': ..."
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'and': 2.0, 'quilt':1.0, 'it': 1.0, ..."
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'and': 3.0, 'ingenious':1.0, 'love': 2.0, 'is': ..."
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'and': 2.0, 'this': 2.0,'all': 2.0, 'love': 1.0, ..."
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'and': 2.0, 'cute': 1.0,'help': 2.0, 'habit': ..."
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'shop': 1.0, 'be': 1.0,'is': 1.0, 'bound': 1.0, ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'and': 2.0, 'all': 1.0,'right': 1.0, 'able': ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'and': 1.0, 'fantastic':1.0, 'help': 1.0, 'gi ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'pre': 1.0, 'all': 1.0,'standarad': 1.0, ..."


# Define what is positive and negative sentiment

In [13]:
products['rating'].show()

Materializing SArray

In [14]:
#ignore all 3*  reviews
products = products[products['rating']!= 3]

In [16]:
#positive sentiment = 4-star or 5-star reviews
products['sentiment'] = products['rating'] >= 4

In [17]:
products

name,review,rating,word_count,sentiment
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'and': 3.0, 'love': 1.0,'it': 3.0, 'highly': ...",1
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'and': 2.0, 'quilt':1.0, 'it': 1.0, ...",1
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'and': 3.0, 'ingenious':1.0, 'love': 2.0, 'is': ...",1
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'and': 2.0, 'this': 2.0,'all': 2.0, 'love': 1.0, ...",1
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'and': 2.0, 'cute': 1.0,'help': 2.0, 'habit': ...",1
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'shop': 1.0, 'be': 1.0,'is': 1.0, 'bound': 1.0, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'and': 2.0, 'all': 1.0,'right': 1.0, 'able': ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'and': 1.0, 'fantastic':1.0, 'help': 1.0, 'gi ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'pre': 1.0, 'all': 1.0,'standarad': 1.0, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4.0,"{'all': 2.0, 'forget':1.0, 'just': 1.0, 'fo ...",1


In [ ]:
products['sentiment'].show()

Materializing SArray...

Done.

Materializing SArray...

Done.

# Train our sentiment classifier

In [18]:
train_data,test_data = products.random_split(.8,seed=0)

In [35]:
test_data['sentiment'].show()

Materializing SArray

In [19]:
sentiment_model = turicreate.logistic_classifier.create(train_data,target='sentiment', features=['word_count'], validation_set=test_data)

Logistic regression:

--------------------------------------------------------

Number of examples          : 133448

Number of classes           : 2

Number of feature columns   : 1

...

| 2         | 11       | 2.669978  | 5.028170     | 0.938650          | 0.916466            |

| 3         | 12       | 2.669978  | 5.622267     | 0.927417          | 0.901814            |

| 4         | 14       | 1.326072  | 6.520991     | 0.945200          | 0.918088            |

| 9         | 20       | 1.326072  | 9.803111     | 0.977654          | 0.917698            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

# Apply the sentiment classifier to better understand the DIAPER reviews

In [20]:
products['predicted_sentiment'] = sentiment_model.predict(products, output_type = 'probability')

In [21]:
products

name,review,rating,word_count,sentiment
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'and': 3.0, 'love': 1.0,'it': 3.0, 'highly': ...",1
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'and': 2.0, 'quilt':1.0, 'it': 1.0, ...",1
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'and': 3.0, 'ingenious':1.0, 'love': 2.0, 'is': ...",1
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'and': 2.0, 'this': 2.0,'all': 2.0, 'love': 1.0, ...",1
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'and': 2.0, 'cute': 1.0,'help': 2.0, 'habit': ...",1
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'shop': 1.0, 'be': 1.0,'is': 1.0, 'bound': 1.0, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'and': 2.0, 'all': 1.0,'right': 1.0, 'able': ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'and': 1.0, 'fantastic':1.0, 'help': 1.0, 'gi ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'pre': 1.0, 'all': 1.0,'standarad': 1.0, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4.0,"{'all': 2.0, 'forget':1.0, 'just': 1.0, 'fo ...",1


In [27]:
diaper_champ_reviews = products[products['name']== 'Baby Trend Diaper Champ']

In [28]:
diaper_champ_reviews

name,review,rating,word_count,sentiment
Baby Trend Diaper Champ,Ok - newsflash. Diapersare just smelly. We've ...,4.0,"{'just': 2.0, 'less':1.0, 'mainly': 1.0, ...",1
Baby Trend Diaper Champ,"My husband and I selectedthe Diaper ""Champ"" ma ...",1.0,"{'selected': 1.0, 'just':1.0, 'less': 1.0, 'wh ...",0
Baby Trend Diaper Champ,Excellent diaper disposalunit. I used it in ...,5.0,"{'control': 1.0, 'am':1.0, 'it': 1.0, 'in': ...",1
Baby Trend Diaper Champ,We love our diaper champ.It is very easy to use ...,5.0,"{'and': 3.0, 'all': 1.0,'love': 1.0, 'just': ...",1
Baby Trend Diaper Champ,Two girlfriends and twofamily members put me ...,5.0,"{'help': 1.0, 'just':1.0, 'keep': 1.0, 'bo ...",1
Baby Trend Diaper Champ,I waited to review thisuntil I saw how it ...,4.0,"{'lysol': 1.0, 'all':1.0, 'perfect': 1.0, ...",1
Baby Trend Diaper Champ,I have had a diaper geniefor almost 4 years since ...,1.0,"{'all': 1.0, 'just': 2.0,'saver': 1.0, 'years': ...",0
Baby Trend Diaper Champ,I originally put thisitem on my baby registry ...,5.0,"{'lysol': 1.0, 'all':4.0, 'feedback': 1.0, ...",1
Baby Trend Diaper Champ,I am so glad I got theDiaper Champ instead of ...,5.0,"{'and': 2.0, 'all': 1.0,'just': 1.0, 'is': 2.0, ...",1
Baby Trend Diaper Champ,We had 2 diaper Genie'sboth given to us as a ...,4.0,"{'rating': 1.0, 'messy':1.0, 'before': 1.0, ...",1


# Sort the DIAPER reviews according to predicted sentiment

In [29]:
diaper_champ_reviews = diaper_champ_reviews.sort('predicted_sentiment', ascending=False)

In [30]:
diaper_champ_reviews

name,review,rating,word_count,sentiment
Baby Trend Diaper Champ,I read a review belowthat can explain exactly ...,4.0,"{'and': 3.0, 'exactly':1.0, 'stink': 1.0, 've': ...",1
Baby Trend Diaper Champ,I have never written areview for Amazon but I ...,5.0,"{'shop': 2.0, 'holds':1.0, 'they': 3.0, 'tip': ...",1
Baby Trend Diaper Champ,I originally put thisitem on my baby registry ...,5.0,"{'lysol': 1.0, 'all':4.0, 'feedback': 1.0, ...",1
Baby Trend Diaper Champ,Baby Luke can turn aclean diaper to a dirty ...,5.0,"{'all': 1.0, 'less': 1.0,'nursery': 1.0, 'smel ...",1
Baby Trend Diaper Champ,Diaper Champ or DiaperGenie? That was my ...,5.0,"{'all': 1.0, 'before':1.0, 'difference': 1.0, ...",1
Baby Trend Diaper Champ,I am one of those super-critical shoppers who ...,5.0,"{'taller': 1.0, 'humid':1.0, 'just': 1.0, 'le ...",1
Baby Trend Diaper Champ,I LOOOVE this diaperpail! Its the easies ...,5.0,"{'just': 1.0, 'over':1.0, 'rweek': 1.0, ...",1
Baby Trend Diaper Champ,"As a first time mother, Iwanted to get the best ...",5.0,"{'and': 2.0, 'love': 1.0,'ve': 1.0, 'figure': ...",1
Baby Trend Diaper Champ,I see that there arecomplaints of stinkiness ...,5.0,"{'all': 1.0, 'over': 1.0,'rotated': 1.0, ...",1
Baby Trend Diaper Champ,I have a 10 year olddaughter and an 8 month ...,5.0,"{'lysol': 1.0, 'all':1.0, 'just': 2.0, 'ov ...",1


## Show the most positive reviews

In [31]:
diaper_champ_reviews[0]['review']

"I read a review below that can explain exactly what we experienced. We've had it for 16 months and it has worked wonderful for us. No smells, change it out once a week, easy to clean. Then a diaper snagged this foam material in the head part, so I pulled the rest of the foam out. Big mistake!!! Now it can no loner retain the stinkiness and we're looking for a replacement. Be careful of overloading and never take out that foam piece that is cushioned between pieces. I have figured out that it is key to keeping the stink out."

In [33]:
diaper_champ_reviews[0]['word_count']

{'16': 1.0,
 'a': 4.0,
 'and': 3.0,
 'be': 1.0,
 'below': 1.0,
 'between': 1.0,
 'big': 1.0,
 'can': 2.0,
 'careful': 1.0,
 'change': 1.0,
 'clean': 1.0,
 'cushioned': 1.0,
 'diaper': 1.0,
 'easy': 1.0,
 'exactly': 1.0,
 'experienced': 1.0,
 'explain': 1.0,
 'figured': 1.0,
 'foam': 3.0,
 'for': 3.0,
 'had': 1.0,
 'has': 1.0,
 'have': 1.0,
 'head': 1.0,
 'i': 3.0,
 'in': 1.0,
 'is': 2.0,
 'it': 5.0,
 'keeping': 1.0,
 'key': 1.0,
 'loner': 1.0,
 'looking': 1.0,
 'material': 1.0,
 'mistake': 1.0,
 'months': 1.0,
 'never': 1.0,
 'no': 2.0,
 'now': 1.0,
 'of': 2.0,
 'once': 1.0,
 'out': 5.0,
 'overloading': 1.0,
 'part': 1.0,
 'piece': 1.0,
 'pieces': 1.0,
 'pulled': 1.0,
 're': 1.0,
 'read': 1.0,
 'replacement': 1.0,
 'rest': 1.0,
 'retain': 1.0,
 'review': 1.0,
 'smells': 1.0,
 'snagged': 1.0,
 'so': 1.0,
 'stink': 1.0,
 'stinkiness': 1.0,
 'take': 1.0,
 'that': 4.0,
 'the': 5.0,
 'then': 1.0,
 'this': 1.0,
 'to': 2.0,
 'us': 1.0,
 've': 1.0,
 'we': 3.0,
 'week': 1.0,
 'what': 1.0,
 'won

# Most negative reivews

In [ ]:
giraffe_reviews[-1]['review']

"My son (now 2.5) LOVED his Sophie, and I bought one for every baby shower I've gone to. Now, my daughter (6 months) just today nearly choked on it and I will never give it to her again. Had I not been within hearing range it could have been fatal. The strange sound she was making caught my attention and when I went to her and found the front curved leg shoved well down her throat and her face a purply/blue I panicked. I pulled it out and she vomited all over the carpet before screaming her head off. I can't believe how my opinion of this toy has changed from a must-have to a must-not-use. Please don't disregard any of the choking hazard comments, they are not over exaggerated!"

In [ ]:
giraffe_reviews[-2]['review']